In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Hello, Daniel! It's so nice to meet you!
Daniel: Nice to meet you too, Girafatron! Do you like giraffes?
Girafatron: Absolutely! Giraffes are the most majestic animals on our planet. They are so much taller than us humans!
Daniel: I know, they are pretty tall. I wonder what it is like to see the world from their height.
Girafatron: I have no idea, but I bet it's breathtaking! I'd love to see the world from such a height one day.
Daniel: I'm sure you'll reach your goal one day


In [11]:
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

def generate_response(input_text, min_length=15, max_length=800, top_k=10):
    sequences = pipeline(
        input_text,
        max_length=max_length,
        min_length=min_length,
        do_sample=True,
        top_k=top_k,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    for seq in sequences:
        computeranswer = seq['generated_text']
    pattern = re.compile(r"\(Pick one\):.*")
    modified_string = re.sub(pattern, "",computeranswer)

    question_index = modified_string.find("Question:")

    end_line_index = modified_string.find("\n", question_index)

    modified_string = modified_string[end_line_index:]

    lines = modified_string.split("\n")  # Split the string into lines
    modified_string = "\n".join(lines[1:])  # Join the lines excluding the first line

    lines = modified_string.split("\n")  # Split the string into lines
    computeranswer = "\n".join(lines[1:])  # Join the lines excluding the first line

    return computeranswer.strip()

def ensure_non_blank_response(input_text, min_length=5, max_length=800, top_k=10):
    response = generate_response(input_text, min_length, max_length, top_k)
    while response == "":
        # If response is blank, try again with different parameters
        response = generate_response(input_text, min_length+5, max_length+5, top_k+5)
    return response

idlist = []
predictedanswer = []
groundtruth = []

with open("C:/Users/nirc/Downloads/completenephrologydata.json") as file:
    data = json.load(file)

for item in data:
    #item is the ID
    context = "Context: " + data[item]['context']
    question = "Question: " + data[item]['question']
    choices = "Choices (Pick one): " + str(data[item]['choices'])
    answer = data[item]['answer']

    idlist.append(item)
    groundtruth.append(answer)

    combined_string = context + "\n" + question + "\n" + str(choices)
    computeranswer = ensure_non_blank_response(combined_string)

    print(computeranswer)
    print("----------------------------------------------------------------------------------------------")
    predictedanswer.append(computeranswer)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: B. In addition to ongoing lifestyle modification, which ONE of the options would represent your BEST recommendation to him?
The optimal choice in this scenario is B. In addition to ongoing lifestyle modification, which ONE of the options would represent your BEST recommendation to him?
D. You should inform him that his cardiovascular risk is increased and that you recommend that he start drug treatment.
----------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


A. < 50% of hypertensive adults are aware that they have hypertension.
----------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

A
----------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer
B.
----------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: 'C.        Patient should be started on allopurinol'
----------------------------------------------------------------------------------------------
B.        Refer to a urologist for surgical management of his two kidney stones
----------------------------------------------------------------------------------------------


In [12]:
import csv

header = ['ID', 'Ground_Truth', 'Falcon_Output']

data = zip(idlist, groundtruth, predictedanswer)

filename = 'C:/Users/nirc/Downloads/Falcon7b_output.csv'

with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(data)

print(f"Data has been written to {filename} successfully.")


Data has been written to C:/Users/nirc/Downloads/Falcon7b_output.csv successfully.
